# 关联分析
从大规模数据集中寻找对象之间的隐含关系被称为关联分析(association analysis)或者关联规则学习(association rule learning)。这里的问题在于寻找不同组合是十分耗时的，所需的计算代价很高，所以需要更智能的方法在合理的时间范围内找到频繁项集
- 关联分析是一种在大规模数据集中寻找有趣关系的任务，其有两种形式
    - 频繁项集(frequent item sets): 经常出现在一块的物品的集合
    - 关联规则(association rules): 暗示了两种物品之间可能存在很强的关系
- 一个项集的支持度(support)被定义为数据集中包含该项集的记录所占比例
- 置信度是针对一条关联规则定义的。定义某个元素出现的条件下另一个元素出现的概率


# Apriori
> Apriori原理是说某个项集是频繁的，那么其子集也是频繁的，反过来一个项集是非频繁的，那么其超集也是非频繁的。

- Aprior等价于a priori表示一个先验。使用该原理可以避免项集指数增长
- Apriori算法是发现频繁项集的一种方法
- 算法流程
    1. 首先生成所有单个物品的项集列表
    2. 扫描交易记录查看哪些项集满足最小支持度要，去掉不满足的项
    3. 对剩下的集合进行组合生成包含两个元素的项集
    4. 重复直到所有项集被去掉

In [1]:
import numpy as np

def loadDataSet():
    return [[1,3,4],[2,3,5],[1,2,3,5],[2,5]]

def createC1(dataSet):
    C1 = []
    for transaction in dataSet:
        for item in transaction:
            if not [item] in C1:
                C1.append([item])
    C1.sort()
    # 对C1中的每个项构建一个不变集合
    return list(map(frozenset,C1))

def scanD(D,Ck,minSupport):
    ssCnt = {}
    for tid in D:
        for can in Ck:
            if can.issubset(tid):
                if can not in ssCnt:
                    ssCnt[can] = 1
                else:
                    ssCnt[can] += 1
    numItems = float(len(D))
    retList = []
    supportData = {}
    for key in ssCnt:
        support = ssCnt[key]/numItems
        if support >= minSupport:
            retList.insert(0,key)
        supportData[key] = support
    return retList,supportData

In [2]:
dataSet = loadDataSet()
dataSet

[[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]

In [3]:
D = list(map(set,dataSet))
D

[{1, 3, 4}, {2, 3, 5}, {1, 2, 3, 5}, {2, 5}]

In [4]:
C1 = createC1(dataSet)
C1

[frozenset({1}),
 frozenset({2}),
 frozenset({3}),
 frozenset({4}),
 frozenset({5})]

In [5]:
L1,suppData0 = scanD(D,C1,0.5)
L1

[frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})]

---

In [6]:
def apriorGen(Lk,k):
    retList = []
    lenLk = len(Lk)
    for i in range(lenLk):
        for j in range(i+1,lenLk):
            L1 = list(Lk[i])[:k-2]
            L2 = list(Lk[j])[:k-2]
            L1.sort()
            L2.sort()
            if L1==L2:
                retList.append(Lk[i] | Lk[j])
    return retList

In [7]:
def apriori(dataSet,minSupport=0.5):
    C1 = createC1(dataSet)
    D = list(map(set,dataSet))
    L1,supportData = scanD(D,C1,minSupport)
    L = [L1]
    k=2
    while (len(L[k-2])>0 ):
        Ck = apriorGen(L[k-2],k)
        Lk,supK = scanD(D,Ck,minSupport)
        supportData.update(supK)
        L.append(Lk)
        k += 1
    return L,supportData

In [8]:
L,suppData = apriori(dataSet)
L

[[frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})],
 [frozenset({2, 3}), frozenset({3, 5}), frozenset({2, 5}), frozenset({1, 3})],
 [frozenset({2, 3, 5})],
 []]

**L包含最小支持度为0.5的频繁项列表**

In [9]:
L,suppData = apriori(dataSet,0.7)
L

[[frozenset({5}), frozenset({2}), frozenset({3})], [frozenset({2, 5})], []]

In [10]:
suppData

{frozenset({1}): 0.5,
 frozenset({3}): 0.75,
 frozenset({4}): 0.25,
 frozenset({2}): 0.75,
 frozenset({5}): 0.75,
 frozenset({2, 5}): 0.75,
 frozenset({3, 5}): 0.5,
 frozenset({2, 3}): 0.5}

# 从频繁项集中挖掘关联规则
- 类似的，如果某条规则不满足最小可信度要求，那么该规则的所有子集也不会满足最小可信度要求
    - 可以首先从一个频繁项集开始，接着创建一个规则列表其中规则右边只包含一个元素，然后对其测试
    - 接下来合并所有剩余规则，规则右边包含两个元素
    - 这种方法称为**分级法**

In [11]:
def calcConf(freqSet,H,supportData,brl,minConf=0.7):
    prunedH = []
    for conseq in H:
        conf = supportData[freqSet]/supportData[freqSet-conseq]
        if conf >= minConf:
            print(freqSet-conseq,'-->',conseq,'conf:',conf)
            brl.append((freqSet-conseq,conseq,conf))
            prunedH.append(conseq)
    return prunedH

def rulesFromConseq(freqSet,H,supportData,brl,minConf=0.7):
    m = len(H[0])
    if (len(freqSet) > (1+m)):
        Hmp1 = apriorGen(H,m+1)
        Hmp1 = calcConf(freqSet,Hmp1,supportData,brl,minConf)
        if (len(Hmp1)>1):
            rulesFromConseq(freqSet,Hmp1,supportData,brl,minConf)

def generateRules(L,supportData,minConf=0.7):
    bigRuleList = []
    for i in range(1,len(L)):
        for freqSet in L[i]:
            H1 = [frozenset([item]) for item in freqSet]
            if (i>1):
                rulesFromConseq(freqSet,H1,supportData,bigRuleList,minConf)
            else:
                calcConf(freqSet,H1,supportData,bigRuleList,minConf)
    return bigRuleList

In [12]:
L,suppData = apriori(dataSet,0.5)
L

[[frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})],
 [frozenset({2, 3}), frozenset({3, 5}), frozenset({2, 5}), frozenset({1, 3})],
 [frozenset({2, 3, 5})],
 []]

In [13]:
rules = generateRules(L,suppData,minConf=0.7)

frozenset({5}) --> frozenset({2}) conf: 1.0
frozenset({2}) --> frozenset({5}) conf: 1.0
frozenset({1}) --> frozenset({3}) conf: 1.0


In [14]:
rules = generateRules(L,suppData,minConf=0.5)

frozenset({3}) --> frozenset({2}) conf: 0.6666666666666666
frozenset({2}) --> frozenset({3}) conf: 0.6666666666666666
frozenset({5}) --> frozenset({3}) conf: 0.6666666666666666
frozenset({3}) --> frozenset({5}) conf: 0.6666666666666666
frozenset({5}) --> frozenset({2}) conf: 1.0
frozenset({2}) --> frozenset({5}) conf: 1.0
frozenset({3}) --> frozenset({1}) conf: 0.6666666666666666
frozenset({1}) --> frozenset({3}) conf: 1.0
frozenset({5}) --> frozenset({2, 3}) conf: 0.6666666666666666
frozenset({3}) --> frozenset({2, 5}) conf: 0.6666666666666666
frozenset({2}) --> frozenset({3, 5}) conf: 0.6666666666666666


# e.g. 发现毒蘑菇的相似特征

In [15]:
mushDatSet = [line.split() for line in open('../data/Apriori/mushroom.dat').readlines()]

In [16]:
L,suppData = apriori(mushDatSet,minSupport=0.3)

In [17]:
for item in L[1]:
    if item.intersection('2'):
        print(item)

frozenset({'28', '2'})
frozenset({'2', '53'})
frozenset({'2', '23'})
frozenset({'34', '2'})
frozenset({'36', '2'})
frozenset({'59', '2'})
frozenset({'2', '63'})
frozenset({'2', '67'})
frozenset({'2', '76'})
frozenset({'2', '85'})
frozenset({'86', '2'})
frozenset({'90', '2'})
frozenset({'93', '2'})
frozenset({'39', '2'})
